## 1. Data Access and Initial Validation

Before building the data pipeline, we first need to confirm that all source files are correctly stored in the Databricks Volume created for this project.  
In this notebook, we will work with four CSV files that were uploaded to the Volume `nba_project`.

This initial step ensures that:
- the Volume path is correct,
- the files are accessible by Spark,
- and the notebook environment is ready for the ETL workflow that follows.

We begin by listing the contents of the Volume to verify the presence of all four CSV files.

In [0]:
display(dbutils.fs.ls("/Volumes/workspace/default/nba_project"))

path,name,size,modificationTime
dbfs:/Volumes/workspace/default/nba_project/1.NBA_Dataset.csv,1.NBA_Dataset.csv,4334640,1764414484000
dbfs:/Volumes/workspace/default/nba_project/2.ListOfMVPs_AllSeasons.csv,2.ListOfMVPs_AllSeasons.csv,2623,1764414473000
dbfs:/Volumes/workspace/default/nba_project/3.Champions.csv,3.Champions.csv,3180,1764414473000
dbfs:/Volumes/workspace/default/nba_project/4.Franchise_Abbrev.csv,4.Franchise_Abbrev.csv,1465,1764414473000


### 1.1 Load the main CSV file

In this step, we will load the file `1.NBA_Dataset.csv` directly from the Databricks Volume.  
The goal is simply to check whether the file can be read correctly, preview the first rows, and confirm the schema (column names and data types).  
If this step works, it means the Volume path is correct and Spark can process the data normally.


In [0]:
# This cell loads Python and Spark functions required throughout the notebook.
from pyspark.sql import functions as F
from pyspark.sql import types as T


In [0]:
#Define path where the file is located
stats_path = "/Volumes/workspace/default/nba_project/1.NBA_Dataset.csv"

#Fetch the content as a dataframe
df_stats = (
  spark.read
    .option("header", True)  #First row as column headers
    .option("inferSchema", True) #Detect data types automatically
    .csv(stats_path)
)

#Preview first 5 rows
display(df_stats.limit(5))

#Display the schema
df_stats.printSchema()

season,player,pos,age,team_id,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg3_per_g,fg3a_per_g,fg3_pct,fg2_per_g,fg2a_per_g,fg2_pct,efg_pct,ft_per_g,fta_per_g,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g,mp,per,ts_pct,fg3a_per_fga_pct,fta_per_fga_pct,orb_pct,drb_pct,trb_pct,ast_pct,stl_pct,blk_pct,tov_pct,usg_pct,ows,dws,ws,ws_per_48,obpm,dbpm,bpm,vorp,award_share,mov,mov_adj,win_loss_pct
1982,Kareem Abdul-Jabbar,C,34,LAL,76,76,35.2,9.9,17.1,0.579,0.0,0.0,0.0,9.9,17.1,0.58,0.579,4.1,5.8,0.706,2.3,6.4,8.7,3.0,0.8,2.7,3.0,2.9,23.9,2677,23.4,0.608,0.002,0.34,7.3,19.1,13.4,11.9,1.1,4.1,13.3,25.6,6.9,3.9,10.7,0.192,3.8,1.2,5.0,4.7,0.045,4.87,4.37,0.695
1982,Alvan Adams,C,27,PHO,79,75,30.3,6.4,13.0,0.494,0.0,0.0,0.0,6.4,13.0,0.494,0.494,2.3,2.9,0.781,1.7,5.7,7.4,4.5,1.4,1.0,2.5,3.4,15.1,2393,18.6,0.529,0.001,0.227,6.6,20.2,13.6,22.1,2.3,1.9,14.8,22.8,2.5,4.7,7.2,0.144,1.4,2.2,3.6,3.4,0.0,3.45,3.05,0.561
1982,Mark Aguirre,SF,22,DAL,51,20,28.8,7.5,16.1,0.465,0.5,1.4,0.352,7.0,14.7,0.475,0.48,3.3,4.8,0.68,1.7,3.1,4.9,3.2,0.7,0.4,2.6,3.0,18.7,1468,17.3,0.514,0.087,0.301,6.7,12.9,9.7,18.6,1.2,0.9,12.7,29.8,1.0,0.8,1.9,0.061,2.3,-1.6,0.7,1.0,0.0,-4.43,-4.48,0.341
1982,Danny Ainge,SG,22,BOS,53,1,10.6,1.5,4.2,0.357,0.1,0.3,0.294,1.4,3.8,0.363,0.369,1.1,1.2,0.862,0.5,0.6,1.1,1.6,0.7,0.1,1.0,1.6,4.1,564,10.1,0.439,0.077,0.294,5.0,5.9,5.5,19.7,3.1,0.3,17.5,21.5,-0.3,0.8,0.5,0.042,-3.7,1.0,-2.7,-0.1,0.0,6.38,6.35,0.768
1982,Tiny Archibald,PG,33,BOS,68,51,31.9,4.5,9.6,0.472,0.1,0.2,0.375,4.4,9.4,0.475,0.477,3.5,4.6,0.747,0.4,1.3,1.7,8.0,0.8,0.0,2.6,1.9,12.6,2167,14.3,0.542,0.025,0.485,1.3,4.5,2.9,31.9,1.1,0.1,18.4,17.9,3.4,1.8,5.2,0.115,1.4,-1.3,0.1,1.1,0.0,6.38,6.35,0.768


root
 |-- season: integer (nullable = true)
 |-- player: string (nullable = true)
 |-- pos: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- team_id: string (nullable = true)
 |-- g: integer (nullable = true)
 |-- gs: integer (nullable = true)
 |-- mp_per_g: double (nullable = true)
 |-- fg_per_g: double (nullable = true)
 |-- fga_per_g: double (nullable = true)
 |-- fg_pct: double (nullable = true)
 |-- fg3_per_g: double (nullable = true)
 |-- fg3a_per_g: double (nullable = true)
 |-- fg3_pct: double (nullable = true)
 |-- fg2_per_g: double (nullable = true)
 |-- fg2a_per_g: double (nullable = true)
 |-- fg2_pct: double (nullable = true)
 |-- efg_pct: double (nullable = true)
 |-- ft_per_g: double (nullable = true)
 |-- fta_per_g: double (nullable = true)
 |-- ft_pct: double (nullable = true)
 |-- orb_per_g: double (nullable = true)
 |-- drb_per_g: double (nullable = true)
 |-- trb_per_g: double (nullable = true)
 |-- ast_per_g: double (nullable = true)
 |-- stl_per_g

### 1.2 Load the remaining CSV files

In this step, we will load the remaining three CSV files from the same Volume:
- the list of MVPs by season,
- the list of NBA champions by season,
- and the franchise–abbreviation reference.

The goal is simply to confirm that all three files can be read, inspect a few sample rows, and check the inferred schemas.  
If everything loads correctly here, we will be ready to save these datasets as tables and create the cleaned views in the next steps.


In [0]:
#Define path where the file is located
mvp_path = "/Volumes/workspace/default/nba_project/2.ListOfMVPs_AllSeasons.csv"
champions_path = "/Volumes/workspace/default/nba_project/3.Champions.csv"
franchise_path = "/Volumes/workspace/default/nba_project/4.Franchise_Abbrev.csv"

# Fetch the content as a dataframes for each file
df_mvp = (
    spark.read
      .option("header", True)  #First row as column headers
      .option("inferSchema", True) #Detect data types automatically
      .csv(mvp_path)
)

df_champions = (
    spark.read
      .option("header", True)  
      .option("inferSchema", True) 
      .csv(champions_path)
)

df_franchise = (
    spark.read
      .option("header", True) 
      .option("inferSchema", True) 
      .csv(franchise_path)
)

#Preview first 5 rows from each file/df
display(df_mvp.limit(5))
display(df_champions.limit(5))
display(df_franchise.limit(5))

# Display the schema for each df
df_mvp.printSchema()
df_champions.printSchema()
df_franchise.printSchema()

YEAR,PLAYER,TEAM
2025,Shai Gilgeous-Alexander,Oklahoma City Thunder
2024,Nikola Jokic,Denver Nuggets
2023,Joel Embiid,Philadelphia 76ers
2022,Nikola Jokic,Denver Nuggets
2021,Nikola Jokic,Denver Nuggets


Season,NBA Champions,Conference
1947,Philadelphia Warriors,null
1948,Baltimore Bullets,null
1949,Minneapolis Lakers,null
1950,Minneapolis Lakers,null
1951,Rochester Royals,null


Franchise,Abbreviation
Atlanta Hawks,ATL
Baltimore Bullets,BLB
Boston Celtics,BOS
Brooklyn Nets,BRK
Buffalo Braves,BUF


root
 |-- YEAR: integer (nullable = true)
 |-- PLAYER: string (nullable = true)
 |-- TEAM: string (nullable = true)

root
 |-- Season: integer (nullable = true)
 |-- NBA Champions: string (nullable = true)
 |-- Conference: string (nullable = true)

root
 |-- Franchise: string (nullable = true)
 |-- Abbreviation: string (nullable = true)



## 2. Save the raw DataFrames as Bronze tables

In this project we follow the Databricks Medallion Architecture, which organises data into three layers: Bronze, Silver and Gold.

- **Bronze** tables contain raw data exactly as sourced, without transformations.
- **Silver** tables will store cleaned and standardised versions of the data.
- **Gold** tables will hold the final analytical datasets used to answer the questions.

In this step, we will save the four DataFrames loaded from the CSV files into the Bronze layer. They will serve as the starting point for all subsequent transformations:
- `nba_stats_bronze` holds the player statistics by season and team,
- `mvp_bronze` holds the list of MVPs by season,
- `champions_bronze` holds the list of NBA champions by season,
- `franchise_bronze` holds the franchise names and their abbreviations.

Saving them as tables makes it easier to query the data later using SQL and to build the cleaned views on top of a consistent base.


In [0]:
# Save each table into Bronze tables
df_stats.write.mode("overwrite").saveAsTable("nba_stats_bronze")
df_mvp.write.mode("overwrite").saveAsTable("mvp_bronze")
df_champions.write.mode("overwrite").saveAsTable("champions_bronze")
df_franchise.write.mode("overwrite").saveAsTable("franchise_bronze")


In [0]:
# Check tables created
spark.sql("SHOW TABLES").show(truncate=False)

+--------+------------------+-----------+
|database|tableName         |isTemporary|
+--------+------------------+-----------+
|default |champions_bronze  |false      |
|default |champions_silver  |false      |
|default |franchise_bronze  |false      |
|default |franchise_silver  |false      |
|default |mvp_baseline_gold |false      |
|default |mvp_bronze        |false      |
|default |mvp_champions_gold|false      |
|default |mvp_silver        |false      |
|default |mvp_stats_gold    |false      |
|default |nba_stats_bronze  |false      |
|default |nba_stats_gold    |false      |
|default |nba_stats_silver  |false      |
+--------+------------------+-----------+



COMMENT: The previous cell’s output includes silver and gold tables in addition to the bronze tables. This occurred because the entire workflow needed to be re-run during debugging, at a point when the silver and gold tables had already been created and persisted in the database. 

## 3. Data Quality Checks

### 3.1 Data quality checks on `nba_stats_bronze`

In this step, we perform a first data quality check on the Bronze table `nba_stats_bronze`.

The goal here is to:
- inspect basic descriptive statistics for the numerical columns (such as minimum and maximum values),
- confirm that the observed ranges make sense for the basketball context (e.g. games played, minutes per game, points per game).


In [0]:
# Load the bronze table as a dataframe
df_stats_bronze = spark.table("nba_stats_bronze")

# Select key numerical columns for inspection
numeric_cols = ["g", "gs", "mp_per_g", "fg_pct", "fg3_pct", "ft_pct", "trb_per_g", "ast_per_g", "stl_per_g", "blk_per_g", "tov_per_g", "pf_per_g", "pts_per_g", "mp"]

# Display statistical summary for the selected columns
df_stats_bronze.select(numeric_cols).summary("count", "min", "max").show(truncate=False)

+-------+-----+-----+--------+------+-------+------+---------+---------+---------+---------+---------+--------+---------+-----+
|summary|g    |gs   |mp_per_g|fg_pct|fg3_pct|ft_pct|trb_per_g|ast_per_g|stl_per_g|blk_per_g|tov_per_g|pf_per_g|pts_per_g|mp   |
+-------+-----+-----+--------+------+-------+------+---------+---------+---------+---------+---------+--------+---------+-----+
|count  |17697|17697|17697   |17634 |15074  |17121 |17697    |17697    |17697    |17697    |17697    |17697   |17697    |17697|
|min    |1    |0    |0.0     |0.0   |0.0    |0.0   |0.0      |0.0      |0.0      |0.0      |0.0      |0.0     |0.0      |0    |
|max    |85   |83   |43.7    |1.0   |1.0    |1.0   |18.7     |14.5     |3.7      |5.6      |5.7      |6.0     |37.1     |3533 |
+-------+-----+-----+--------+------+-------+------+---------+---------+---------+---------+---------+--------+---------+-----+



#### 3.1.1 Investigate anomalies where `g` or `gs` exceed 82

In the previous step, the descriptive statistics revealed that the maximum values of `g` (games played) and `gs` (games started) were higher than the NBA regular-season limit of 82 games.  
This step isolates all records where these anomalies occur, allowing us to understand whether they are data errors, duplicated entries, or a consequence of how the dataset was compiled.

This investigation is part of the data-quality analysis required before defining the cleaning rules for the Silver layer.


In [0]:
%sql
SELECT season, player, team_id, g, gs  
FROM nba_stats_bronze
WHERE g > 82 OR gs > 82
ORDER BY season, player, g DESC;

season,player,team_id,g,gs
1982,Mike Mitchell,TOT,84,83
1982,Mike Woodson,TOT,83,74
1982,Phil Hubbard,TOT,83,40
1983,Clemon Johnson,TOT,83,11
1986,James Donaldson,TOT,83,78
1988,Mark West,TOT,83,41
1988,Tyrone Corbin,TOT,84,5
1992,Thurl Bailey,TOT,84,18
1993,J.R. Reid,TOT,83,25
1996,Benoit Benjamin,TOT,83,71


#### 3.1.2 Outcome of investigation

Investigation revealed that all such cases correspond to records where the team_id field was set to TOT. This label indicates that the player appeared for multiple teams within the same season.

These anomalies will be addressed in the Silver layer by capping their values at 82 (the NBA regular-season limit).

#### 3.1.3 Null-value inspection of `nba_stats_bronze`

In this step we assess the presence of missing (null) values in the Bronze table `nba_stats_bronze`.

Null values can indicate incomplete records, ingestion issues, or gaps in the original data source. Identifying them at the Bronze stage allows us to decide whether they must be cleaned, imputed, or simply acknowledged in the Silver layer.

We will compute the number of null values for every column in the dataset.

In [0]:
# Create an empty dictionary to store results
null_counts_dict = {}

# Loop through columns and count nulls
for column in df_stats_bronze.columns:
  null_count = df_stats_bronze.filter(F.col(column).isNull()).count()
  null_counts_dict[column] = null_count

# Convert the dictionary to a Spark dataframe for display
null_counts_df = spark.createDataFrame(
  [(k,v) for k, v in null_counts_dict.items()],
  ["column", "null_count"] 
)

display(null_counts_df)

column,null_count
season,0
player,0
pos,0
age,0
team_id,0
g,0
gs,0
mp_per_g,0
fg_per_g,0
fga_per_g,0


#### 3.1.4 Count null `fg3_pct` values per season

In the previous null-value inspection we identified several attributes containing missing values.  
Among these, `fg3_pct` showed a particularly high number of null entries.

For didactic purposes, the next step focuses on a quick investigation of how the null `fg3_pct` values are distributed across seasons. Understanding whether these missing values are concentrated in specific years or spread across the dataset helps clarify whether the issue is structural or statistical (e.g., players with zero 3-point attempts).

Due to time limitations, a deeper investigation of all other attributes with null values will not be performed here.  

In [0]:
# Count null values of fg3_pct per season
fg3_nulls_by_season = (
    df_stats_bronze
    .groupBy("season")
    .agg(F.sum(F.col("fg3_pct").isNull().cast("int")).alias("null_fg3_pct"))
    .orderBy("season")
)

display(fg3_nulls_by_season)

season,null_fg3_pct
1982,67
1983,52
1984,65
1985,58
1986,61
1987,58
1988,57
1989,56
1990,60
1991,59


#### 3.1.5 Validate whether null `fg3_pct` values correspond to zero 3-point attempts

After observing that `fg3_pct` contains many null values across all seasons, we now verify whether these nulls are statistically justified.

A player’s 3-point percentage should be null if and only if the player attempted zero 3-point field goals during the season.  
Therefore, in this step we check for any cases where:

- `fg3_pct` is null, **but**
- `fg3a_per_g` (3-point attempts per game) is greater than zero.

If such cases exist, they may indicate data-quality issues in the original dataset.  
If no such cases are found, the null values can be considered valid results of undefined percentages.

In [0]:
%sql
SELECT player, season, fg3_pct, fg3a_per_g  
FROM nba_stats_bronze
WHERE fg3_pct IS NULL
    AND fg3a_per_g >0
ORDER BY season, player;

player,season,fg3_pct,fg3a_per_g


COMMENTS: 
The validation performed using SQL returned no rows where `fg3_pct` was null while `fg3a_per_g` was greater than zero.
This confirms that all missing values of 3-point percentage occur exclusively in cases where the player attempted no 3-point field goals during the season.
Therefore, the null values in `fg3_pct` are considered statistically appropriate and do not represent data-quality issues.
These values will be preserved in the Silver layer without imputation.

#### 3.1.6 Data inspection on categorical columns

We will now examine the categorical attributes in the dataset, focusing on:

- `pos` (player position), and  
- `team_id` (team identifier).

The purpose of this step is to identify possible inconsistencies, unexpected values, or formatting issues.  

Understanding how categorical values are distributed helps define the cleaning rules to be applied in the Silver layer, where standardisation and validation will occur.

In [0]:
%sql
-- List the distinct values for "pos" column.
SELECT DISTINCT pos
 FROM nba_stats_bronze
 ORDER BY pos;

pos
C
C-PF
PF
PF-C
PF-SF
PG
PG-SF
PG-SG
SF
SF-C


COMMENTS: The pos attribute contains both single-position labels (PG, SG, SF, PF, C) and hybrid combinations such as SG-PG, PF-C or SG-PG-SF.
Hybrid labels represent players who operate across multiple on-court roles, which is meaningful from a basketball perspective.
However, to simplify the analytical steps in the Silver and Gold layers, these hybrid labels will be normalised to the primary listed position (i.e., the first component before the hyphen).


In [0]:
%sql
-- List the distinct values for "team_id" column.
SELECT DISTINCT team_id
 FROM nba_stats_bronze
 ORDER BY team_id;

team_id
ATL
BOS
BRK
CHA
CHH
CHI
CHO
CLE
DAL
DEN


#### 3.1.7 Player name readability

Besides numerical ranges and null values, it is also important to verify whether textual attributes are stored in a readable and consistent format.

In this step, we focus on the `player` column of `nba_stats_bronze` and search for names that contain unexpected characters outside a basic ASCII range (letters, spaces, apostrophes and hyphens).  
Names containing unusual symbols may indicate character-encoding issues during CSV export or ingestion. This is relevant as there are a large number of players from countries that use different characters as part of their alphabet.

In [0]:
%sql
-- The query below lists all distinct player names that include at least one non-standard character, so that potential problems can be inspected manually.
SELECT DISTINCT player
FROM nba_stats_bronze
WHERE player RLIKE '[^A-Za-z .''-]'
ORDER BY player;

player
Aleksandar Radojević
Alen Smailagić
Alexis Ajinça
Alperen Şengün
Amar'e Stoudemire
Anderson Varejão
Andris Biedriņš
Andrés Guibert
Andrés Nocioni
Ante Žižić


COMMENTS: The readability check confirmed that all player names are correctly encoded and appear with proper accented characters when processed in Databricks.  
Since the original data is stored correctly, no encoding correction is required in the Silver layer.

### 3.2 Data quality checks on `mvp_bronze`
In this step we perform a simple data quality check on the Bronze table `mvp_bronze`, which contains the list of MVPs by season.

Because this dataset has only three attributes (`YEAR`, `PLAYER`, `TEAM`) and is relatively small and curated, a light inspection is sufficient at this stage.  

We will:
- check the total number of rows, and  
- count null values for each column.

In [0]:
%sql
SELECT COUNT(*) as total_rows,
SUM(CASE WHEN YEAR IS NULL THEN 1 ELSE 0 END) as null_year,
SUM(CASE WHEN PLAYER IS NULL THEN 1 ELSE 0 END) as null_player,
SUM(CASE WHEN TEAM IS NULL THEN 1 ELSE 0 END) as null_team
FROM mvp_bronze;

total_rows,null_year,null_player,null_team
70,0,0,0


COMMENTS: The completeness check shows that the `mvp_bronze` table contains no null values in any of its attributes.  
This indicates that the dataset is fully populated and does not require additional cleaning before moving to the Silver layer.

### 3.3 Data quality check on `champions_bronze`

This dataset contains the NBA champion team for each season.  
Because it is a small and curated dataset, a simple quality check is sufficient at this stage.

The inspection includes:
- counting the number of rows,
- checking for null values across all attributes,
- and verifying that each season appears only once.

These checks help confirm whether the dataset is consistent and ready for the Silver layer.

In [0]:
%sql
-- Check for null values across all columns
SELECT COUNT(*) AS Total_rows,
SUM(CASE WHEN Season IS NULL THEN 1 ELSE 0 END) as null_season,
SUM(CASE WHEN 'NBA Champions' IS NULL THEN 1 ELSE 0 END) as null_champion,
SUM(CASE WHEN Conference IS NULL THEN 1 ELSE 0 END) as null_conference
FROM champions_bronze;

Total_rows,null_season,null_champion,null_conference
79,0,0,23


In [0]:
%sql
-- List the 23 null_conference records for further analysis
SELECT Season, COUNT(*) AS null_conference
FROM champions_bronze
WHERE Conference IS NULL
GROUP BY Season
ORDER BY Season;

Season,null_conference
1947,1
1948,1
1949,1
1950,1
1951,1
1952,1
1953,1
1954,1
1955,1
1956,1


COMMENTS: The quality check identified 23 null values in the `Conference` column, all corresponding to seasons from 1947 to 1969.  
This is expected because the NBA did not use the modern East/West conference structure during these early years.  
Therefore, these null values are historically accurate rather than data-quality issues.
Anyway, the primary statistics dataset (`nba_stats_bronze`) covers seasons from 1982 onwards.  
Because of this, data from earlier seasons in supporting tables (such as `champions_bronze`) falls outside the analytical scope of this project.  
Therefore, null values in early-season attributes such as `Conference` are historically appropriate and require no cleaning.

In [0]:
%sql
-- Confirm that each season only appears once in the table
SELECT Season, COUNT(*) AS num_rows
FROM champions_bronze
GROUP BY Season
HAVING COUNT(*) > 1;

Season,num_rows


COMMENTS: No season appears more than once in the dataset.

### 3.4 Data quality check on `franchise_bronze`

This table contains the official names of NBA franchises and their corresponding abbreviations.  
Because the table is small and well-defined, only a light quality inspection is required.

In this step, we check:
- whether any of the attributes contain null values,
- whether any abbreviations or franchise names are duplicated.

This ensures that the lookup table is consistent before it is used in the Silver layer.

In [0]:
%sql
-- Count null records
SELECT COUNT(*) AS total_rows,
SUM(CASE WHEN Franchise IS NULL THEN 1 ELSE 0 END) AS null_franchise,
SUM(CASE WHEN Abbreviation IS NULL THEN 1 ELSE 0 END) AS null_abbreviation
FROM franchise_bronze;

total_rows,null_franchise,null_abbreviation
63,0,0


In [0]:
%sql
-- Check for Franchise duplications
SELECT *
FROM franchise_bronze
WHERE Franchise IN (
  SELECT Franchise
  FROM franchise_bronze
  GROUP BY Franchise
  HAVING COUNT(*) > 1
);

Franchise,Abbreviation
New Orleans Hornets,NOH
New Orleans Hornets,NOK


COMMENTS: The `franchise_bronze` table contains a duplicated franchise name: *New Orleans Hornets*, associated with both `NOH` and `NOK`.  
This duplication is historically accurate: the team competed as the **New Orleans/Oklahoma City Hornets (NOK)** during the temporary relocation caused by Hurricane Katrina (2005–2007).  
Because both abbreviations are valid, no correction is required, and both codes will be preserved later in the Silver layer.

In [0]:
%sql
-- Check for Abbreviation duplications
SELECT Abbreviation, COUNT(*) AS count_rows
FROM franchise_bronze
GROUP BY Abbreviation
HAVING COUNT(*) > 1;

Abbreviation,count_rows


COMMENTS: No issues found for the Abbreviation column.

## 4. Create Silver tables

### 4.1 Create `nba_stats_silver`

The Bronze table `nba_stats_bronze` contains more than fifty attributes, including many advanced and highly granular metrics.  
For the purposes of this academic project, the main objective is to demonstrate the ability to build and manage a cloud-based data pipeline, rather than to exploit every single statistic available.

To make the subsequent analysis simpler and more transparent, the Silver table will retain only a subset of attributes that are:

- sufficient to characterise player performance,
- directly useful for answering the project questions,
- and easy to interpret in the final report.

The selected columns include:
- identification and context (season, player, team, age, position),
- basic per-game statistics (games, minutes, points, rebounds, assists, steals, blocks, turnovers),
- key shooting percentages (field goal, 3-point and free-throw),
- a small set of advanced metrics (PER, true shooting percentage, Win Shares, 
award share).

In [0]:
%sql
CREATE OR REPLACE TABLE nba_stats_silver AS
SELECT
    -- Season identifier: we keep the original season as the main time dimension
    season,
    
    -- Player name: kept as is. It has already been validated as correct in the Bronze layer
    player AS player_name,
    
    -- Position normalisation:
    -- The original `pos` field may contain hybrid values such as 'SG-PG', 'PF-C' or 'SG-PG-SF'.
    -- To simplify downstream analysis (for example, counting MVPs per position),
    -- we keep only the primary position, defined as the substring before the first hyphen.
    -- Examples:
    --   'SG-PG-SF' -> 'SG'
    --   'PF-C'     -> 'PF'
    --   'C'        -> 'C'     (unchanged when there is no hyphen)
    substring_index(pos, '-', 1) AS pos,
    
    -- Basic context attributes
    age,
    team_id AS team_abbrev,
    
    -- Game-count cleaning:
    -- During the Bronze-layer quality checks, a few records showed `g` (games played)
    -- and `gs` (games started) greater than 82.
    -- As the NBA regular season is limited to 82 games, these values are not realistic.
    -- In the Silver table, we will cap them at 82 to enforce a valid domain constraint.
    -- This does not change the overall structure of the data, but prevents
    -- impossible values from affecting basic statistics.
    CASE WHEN g  > 82 THEN 82 ELSE g  END AS g,
    CASE WHEN gs > 82 THEN 82 ELSE gs END AS gs,
    
    -- Minutes per game: kept as-is, used as a simple measure of playing time
    mp_per_g,
    
    -- Core per-game box score statistics:
    -- These attributes make it possible to describe player performance
    -- without relying on the full set of advanced metrics from the Bronze table.
    trb_per_g,   -- total rebounds per game
    ast_per_g,   -- assists per game
    stl_per_g,   -- steals per game
    blk_per_g,   -- blocks per game
    tov_per_g,   -- turnovers per game
    pts_per_g,   -- points per game
    
    -- Shooting efficiency:
    -- Field goal, 3-point and free-throw percentages provide a compact view
    -- of how efficient the player is as a scorer.
    fg_pct,
    fg3_pct,
    ft_pct,
    
    -- Selected advanced metrics:
    -- A limited set of advanced statistics is carried over to the Silver table
    -- to support more refined comparisons (e.g., between MVPs and non-MVPs)
    -- without overwhelming the analysis with too many variables.
    per,         -- Player Efficiency Rating
    ts_pct,      -- True Shooting percentage
    ws,          -- Win Shares
    award_share  -- Share of MVP voting
FROM nba_stats_bronze;


num_affected_rows,num_inserted_rows


### 4.2 Create `mvp_silver`

The Bronze table `mvp_bronze` contains the list of MVP winners by season, including years that fall outside the scope of the main statistics dataset.

Because the `nba_stats_bronze` table only covers seasons from 1982 to 2022, the Silver-layer MVP table will be restricted to the same time range.  
This ensures that all seasons in `mvp_silver` can be directly matched to player statistics in `nba_stats_silver`.

In addition, the column names are standardised to a consistent naming style.

In [0]:
%sql
CREATE or REPLACE TABLE mvp_silver AS
SELECT 
  YEAR AS season, --Align column name with the main stats table
  PLAYER AS mvp_player_name,
  TEAM AS mvp_team_name
FROM mvp_bronze
WHERE YEAR >= 1982 AND YEAR <=2022; 

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Display recently created table, for verification 
SELECT * 
FROM mvp_silver
LIMIT 5;

season,mvp_player_name,mvp_team_name
2022,Nikola Jokic,Denver Nuggets
2021,Nikola Jokic,Denver Nuggets
2020,Giannis Antetokounmpo,Milwaukee Bucks
2019,Giannis Antetokounmpo,Milwaukee Bucks
2018,James Harden,Houston Rockets


#### 4.2.1 Check name consistency
Before using the MVP data in the Gold layer, it is important to verify that all MVP player names can be successfully matched to the season statistics in `nba_stats_silver`.

In [0]:
%sql
--Identify any MVP record that does not find a corresponding row in `nba_stats_silver`.

SELECT m.season, m.mvp_player_name
FROM mvp_silver as m
LEFT JOIN nba_stats_silver as s
  ON m.season = s.season
  AND m.mvp_player_name = s.player_name
WHERE s.player_name IS NULL
ORDER BY m.season;

season,mvp_player_name
2021,Nikola Jokic
2022,Nikola Jokic


In [0]:
%sql
--Check player name spelling in the nba_stats_silver table
SELECT DISTINCT player_name
FROM nba_stats_silver
WHERE season IN (2021,2022)
  AND player_name ILIKE '%Joki%'
ORDER BY player_name;

player_name
Nikola Jokić


COMMENTS: It's been identified that the MVP dataset stores the player `Nikola Jokic` without an accent, while the statistics dataset uses the form `Nikola Jokić`.  
To ensure a consistent join key across all Silver tables, we need to rebuild `mvp_silver` with a small name correction that standardises this MVP name to the same format used in `nba_stats_silver`.



####4.2.2 Fix inconsistencies identified
Instead of recreating the entire Silver table, a targeted `UPDATE` is applied to `mvp_silver` to standardise the inconsistent player name.  
This change aligns the MVP records with the naming convention used in `nba_stats_silver` and allows all seasons from 1982 to 2022 to be joined correctly later in the Gold layer.

In [0]:
%sql
UPDATE mvp_silver
SET mvp_player_name = 'Nikola Jokić'
WHERE mvp_player_name = 'Nikola Jokic';

num_affected_rows
2


In [0]:
%sql
--Run the name check again to ensure it has been fixed
SELECT
    m.season,
    m.mvp_player_name
FROM mvp_silver m
LEFT JOIN nba_stats_silver s
  ON m.season      = s.season
 AND m.mvp_player_name = s.player_name
WHERE s.player_name IS NULL
ORDER BY m.season;


season,mvp_player_name


COMMENTS: As no rows are displayed as a result of this query, we can confirm all players from the `mvp_silver` table now find a correspondent record in the `nba_stats_silver` table.


### 4.3 Create `champions_silver`
The Bronze table `champions_bronze` contains NBA champions for seasons starting well before the main statistics dataset.  
However, the core analysis in this project is restricted to seasons between 1982 and 2022, matching the coverage of `nba_stats_silver`.

In this step, we build the `champions_silver` table by:
- filtering the data to keep only seasons from 1982 to 2022, and
- standardising the column names to match the naming style used in the other Silver tables.

The `Conference` attribute is preserved as provided in the source data.

In [0]:
%sql
describe champions_bronze;

col_name,data_type,comment
Season,int,null
NBA Champions,string,null
Conference,string,null


In [0]:
%sql
CREATE or REPLACE TABLE champions_silver AS
SELECT
  Season as season,
  `NBA Champions` as champion_team_name,
  Conference as conference
FROM champions_bronze
WHERE Season BETWEEN 1982 and 2022;

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Display recently created table, for verification 
SELECT *
FROM champions_silver
LIMIT 5;

season,champion_team_name,conference
1982,Los Angeles Lakers,Western Conference NBA
1983,Philadelphia 76ers,Eastern Conference NBA
1984,Boston Celtics,Eastern Conference NBA
1985,Los Angeles Lakers,Western Conference NBA
1986,Boston Celtics,Eastern Conference NBA


In [0]:
%sql
-- Check whether the min and max seasons are correct
SELECT MIN(season) as min_season, MAX(season) as max_season
FROM champions_silver;

min_season,max_season
1982,2022


COMMENTS: The table includes only seasons between 1982 and 2022 as expected.

### 4.4 Create `franchise_silver`
The `franchise_bronze` table contains franchise names and their corresponding abbreviations.  
In the Silver layer, this dataset is used as a simple lookup table to support any joins or validations involving team abbreviations.

In this step, we:
- standardise the column names to `franchise_name` and `team_abbrev`, and
- keep all rows from the Bronze table, including historically valid cases where the same franchise appears with different abbreviations (for example, `New Orleans Hornets` with `NOH` and `NOK`).

In [0]:
%sql
CREATE or REPLACE TABLE franchise_silver AS
SELECT
  Franchise as franchise_name,
  Abbreviation as team_abbrev
FROM franchise_bronze;

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Display recently created table
SELECT *
FROM franchise_silver
LIMIT 5;

franchise_name,team_abbrev
Atlanta Hawks,ATL
Baltimore Bullets,BLB
Boston Celtics,BOS
Brooklyn Nets,BRK
Buffalo Braves,BUF


## 5. Create Gold tables

### 5.1 Create `mvp_stats_gold` (MVP + season statistics)

The first Gold-layer object combines the MVP information with the corresponding season-long player statistics.

Using the Silver tables:
- `mvp_silver`, and
- `nba_stats_silver`,

we build a view that contains, for each season between 1982 and 2022:
- who the MVP was,
- their position, team and age,
- their statistics,

This integrated view will later be used to answer questions such as:
- which statistics are most common among MVPs,
- the distribution of MVPs by position,
- and whether certain teams host MVPs more frequently.

In [0]:
%sql
CREATE OR REPLACE TABLE mvp_stats_gold AS
SELECT  
  s.season, 
  s.player_name, 
  s.pos, 
  s.age,
  s.team_abbrev AS stats_team_abbrev,
  m.mvp_team_name AS mvp_team_name,
  s.mp_per_g,
  s.pts_per_g,
  s.trb_per_g,
  s.ast_per_g,
  s.stl_per_g,
  s.blk_per_g,
  s.tov_per_g,
  s.fg_pct,
  s.fg3_pct,
  s.ft_pct,
  s.per,
  s.ts_pct,
  s.ws,
  s.award_share
  FROM nba_stats_silver as s  
  JOIN mvp_silver as m  
  ON s.season = m.season AND s.player_name = m.mvp_player_name;




num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Display recently created table for verification
SELECT *
FROM mvp_stats_gold
LIMIT 5;

season,player_name,pos,age,stats_team_abbrev,mvp_team_name,mp_per_g,pts_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,fg_pct,fg3_pct,ft_pct,per,ts_pct,ws,award_share
1982,Moses Malone,C,26,HOU,Houston Rockets,42.0,31.1,14.7,1.8,0.9,1.5,3.6,0.519,0.0,0.762,26.8,0.576,15.4,0.735
1983,Moses Malone,C,27,PHI,Philadelphia 76ers,37.5,24.5,15.3,1.3,1.1,2.0,3.4,0.501,0.0,0.761,25.1,0.578,15.1,0.96
1984,Larry Bird,PF,27,BOS,Boston Celtics,38.3,24.2,10.1,6.6,1.8,0.9,3.0,0.492,0.247,0.888,24.2,0.552,13.6,0.858
1985,Larry Bird,SF,28,BOS,Boston Celtics,39.5,28.7,10.5,6.6,1.6,1.2,3.1,0.522,0.427,0.882,26.5,0.585,15.7,0.978
1986,Larry Bird,SF,29,BOS,Boston Celtics,38.0,25.8,9.8,6.8,2.0,0.6,3.2,0.496,0.423,0.896,25.6,0.58,15.8,0.981


In [0]:
%sql
-- Confirm how many rows are in the new table (should have 41 concerning years 1982-2022)
SELECT
  COUNT(*)               AS total_rows,
  COUNT(DISTINCT season) AS distinct_seasons
FROM mvp_stats_gold;


total_rows,distinct_seasons
41,41


COMMENTS: The `mvp_stats_gold` table contains the expected number of rows, corresponding to seasons between 1982 and 2022.  

### 5.2 Create `mvp_champions_gold` (compare MVP and champion teams)

This Gold-level table combines:
- the MVP statistics from `mvp_stats_gold`, and
- the season champions from `champions_silver`.

For each season between 1982 and 2022, the table shows:
- who the MVP was,
- which team they played for,
- which team won the NBA championship,
- and a simple flag indicating whether the MVP's team was also the champion in that season.

This table will later be used to answer how often the MVP and the NBA champion come from the same team.

In [0]:
%sql
CREATE OR REPLACE TABLE mvp_champions_gold AS
SELECT 
  m.season,
  m.player_name,
  m.pos,
  m.age,
  m.stats_team_abbrev,
  m.mvp_team_name,
  c.champion_team_name,
--Flag as 1 if MVP's team won the title or 0 otherwise
  CASE WHEN m.mvp_team_name = c.champion_team_name THEN 1 ELSE 0
  END AS mvp_and_champion_same_team
FROM mvp_stats_gold as m
JOIN champions_silver as c
ON m.season = c.season;

num_affected_rows,num_inserted_rows


In [0]:
%sql
-- Display recently created table
SELECT * FROM mvp_champions_gold
LIMIT 3;

season,player_name,pos,age,stats_team_abbrev,mvp_team_name,champion_team_name,mvp_and_champion_same_team
1982,Moses Malone,C,26,HOU,Houston Rockets,Los Angeles Lakers,0
1983,Moses Malone,C,27,PHI,Philadelphia 76ers,Philadelphia 76ers,1
1984,Larry Bird,PF,27,BOS,Boston Celtics,Boston Celtics,1


### 5.3 Create `nba_stats_gold` (main analytical fact table)

Although `nba_stats_silver` already contains a cleaned and simplified version of the player statistics, the analytical queries in this project will be based on the Gold layer only.

To support this design, the table `nba_stats_gold` is created as the main fact table for the analysis layer, using the Silver statistics table as its source. 

No additional transformation is applied at this point. The purpose is to provide a clearly defined, analysis-ready dataset in the Gold layer.

In [0]:
%sql
CREATE OR REPLACE TABLE nba_stats_gold AS
SELECT * FROM nba_stats_silver;


num_affected_rows,num_inserted_rows


In [0]:
%sql
--Display recently created table
SELECT * FROM nba_stats_gold
LIMIT 3;


season,player_name,pos,age,team_abbrev,g,gs,mp_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pts_per_g,fg_pct,fg3_pct,ft_pct,per,ts_pct,ws,award_share
1982,Kareem Abdul-Jabbar,C,34,LAL,76,76,35.2,8.7,3.0,0.8,2.7,3.0,23.9,0.579,0.0,0.706,23.4,0.608,10.7,0.045
1982,Alvan Adams,C,27,PHO,79,75,30.3,7.4,4.5,1.4,1.0,2.5,15.1,0.494,0.0,0.781,18.6,0.529,7.2,0.0
1982,Mark Aguirre,SF,22,DAL,51,20,28.8,4.9,3.2,0.7,0.4,2.6,18.7,0.465,0.352,0.68,17.3,0.514,1.9,0.0


### 5.4 Data summary for minimum and maximum value extraction

The following command uses the `.describe()` function to generate basic statistical information for each gold table.  
This includes the minimum and maximum values, which will be used in the data catalogue.


In [0]:
# Parse gold tables as spark tables
nba_gold_df = spark.table("nba_stats_gold")
mvp_gold_df = spark.table("mvp_stats_gold")
mvp_champions_gold_df = spark.table("mvp_champions_gold")

# Display symmary statistics, including minimum and maximum values
display(nba_gold_df.describe())
display(mvp_gold_df.describe())
display(mvp_champions_gold_df.describe())


summary,season,player_name,pos,age,team_abbrev,g,gs,mp_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pts_per_g,fg_pct,fg3_pct,ft_pct,per,ts_pct,ws,award_share
count,17697,17697,17697,17697,17697,17697,17697,17697,17697,17697,17697,17697,17697,17697,17634,15074,17121,17694,17642,17697,17697
mean,2003.7858394078091,null,null,26.64948861388936,null,53.47488274848845,26.039837260552638,20.40508560773015,3.5970220941402893,1.9071537548737112,0.6709046731084346,0.4213539017912742,1.248680567327781,8.415528055602651,0.4423701372348848,0.260761310866394,0.7264630570644196,12.778682039109299,0.5129243283074453,2.661343730575794,0.0060224331807651015
stddev,11.658167968790812,null,null,4.107071550902049,null,25.291343837149693,29.38856542485869,10.111089115191767,2.5521060987645745,1.8784235522144819,0.4730119727996503,0.5048633483241521,0.8236869548260332,6.100946018426273,0.09341661850343608,0.16809661403807288,0.14114958983465098,6.154654190593995,0.09187031177175772,3.016477177783249,0.059276840243213405
min,1982,A.C. Green,C,18,ATL,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-90.6,0.0,-2.1,0.0
max,2022,Željko Rebrača,SG,44,WSB,82,82,43.7,18.7,14.5,3.7,5.6,5.7,37.1,1.0,1.0,1.0,133.8,1.5,21.2,1.0


summary,season,player_name,pos,age,stats_team_abbrev,mvp_team_name,mp_per_g,pts_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,fg_pct,fg3_pct,ft_pct,per,ts_pct,ws,award_share
count,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41,41
mean,2002.0,null,null,27.634146341463413,null,null,37.36341463414635,27.0,8.88048780487805,6.358536585365852,1.568292682926829,1.0780487804878052,3.253658536585366,0.5093902439024391,0.29417073170731706,0.813048780487805,28.03170731707317,0.5959512195121951,15.895121951219519,0.8964634146341466
stddev,11.979148550710939,null,null,2.7996079864953916,null,null,2.548014530723969,3.7453304260105007,3.4094881956237306,2.7840595907555663,0.5654374520626679,0.8881194162729333,0.5835655751394225,0.03811422178713641,0.13679910497496398,0.08067711919110405,2.9107936223983866,0.034353275840530995,2.7262567028356104,0.08648008370745708
min,1982,Allen Iverson,C,22,BOS,Boston Celtics,30.4,15.5,3.3,1.3,0.5,0.1,2.1,0.42,0.0,0.524,22.0,0.518,9.6,0.691
max,2022,Tim Duncan,SG,35,UTA,Utah Jazz,42.0,35.0,15.3,12.8,3.2,3.7,5.4,0.583,0.454,0.921,32.8,0.669,21.2,1.0


summary,season,player_name,pos,age,stats_team_abbrev,mvp_team_name,champion_team_name,mvp_and_champion_same_team
count,41,41,41,41,41,41,41,41
mean,2002.0,null,null,27.634146341463413,null,null,null,0.34146341463414637
stddev,11.979148550710939,null,null,2.7996079864953916,null,null,null,0.48009145470217307
min,1982,Allen Iverson,C,22,BOS,Boston Celtics,Boston Celtics,0
max,2022,Tim Duncan,SG,35,UTA,Utah Jazz,Toronto Raptors,1


##6. Analysis - answering the key questions
This section presents the analytical component of the project, where the questions defined in the problem statement are addressed using the curated Gold-layer datasets.

Each question is supported by SQL queries and followed by a short interpretation based on the results.

The analysis will answer the following key questions:

1. **Which statistics most consistently appear among MVP winners across seasons?**  
2. **What is the distribution of player positions among MVP winners?**  
3. **Do some teams historically concentrate more MVP awards?**  
4. **How often does the MVP come from the team that wins the NBA championship in the same season?**  
5. **Which players stood out statistically even though they never won the MVP award?**

The following sections address each question step-by-step.

### 6.1 Question: _Which statistics most consistently appear among MVP winners across seasons?_

####6.1.1 Overview and queries
Before identifying which performance metrics are most recurrent among MVP winners, we first compute simple summary statistics (minimum, maximum and average) for a selected group of attributes in the `mvp_stats_gold` table.

This step provides a numerical overview of how MVPs typically perform in areas such as scoring, rebounding, playmaking, defence and shooting efficiency.  
These aggregated values will support the interpretation of which metrics appear consistently strong or stable across MVP seasons, forming the basis for answering the question above.

In [0]:
%sql
SELECT
MIN(pts_per_g) AS min_pts_per_g,
AVG(pts_per_g) AS avg_pts_per_g,
MAX(pts_per_g) AS max_pts_per_g,

MIN(trb_per_g) AS min_trb_per_g,
AVG(trb_per_g) AS avg_trb_per_g,
MAX(trb_per_g) AS max_trb_per_g,

MIN(ast_per_g) AS min_ast_per_g,
AVG(ast_per_g) AS avg_ast_per_g,
MAX(ast_per_g) AS max_ast_per_g,

MIN(stl_per_g) AS min_stl_per_g,
AVG(stl_per_g) AS avg_stl_per_g,
MAX(stl_per_g) AS max_stl_per_g,

MIN(blk_per_g) AS min_blk_per_g,
AVG(blk_per_g) AS avg_blk_per_g,
MAX(blk_per_g) AS max_blk_per_g,

MIN(tov_per_g) AS min_tov_per_g,
AVG(tov_per_g) AS avg_tov_per_g,
MAX(tov_per_g) AS max_tov_per_g,

MIN(fg_pct) AS min_fg_pct,
AVG(fg_pct) AS avg_fg_pct,
MAX(fg_pct) AS max_fg_pct,

MIN(fg3_pct) AS min_fg3_pct,
AVG(fg3_pct) AS avg_fg3_pct,
MAX(fg3_pct) AS max_fg3_pct,

MIN(ft_pct) AS min_ft_pct,
AVG(ft_pct) AS avg_ft_pct,
MAX(ft_pct) AS max_ft_pct,

MIN(per) AS min_per,
AVG(per) AS avg_per,
MAX(per) AS max_per,

MIN(ts_pct) AS min_ts_pct,
AVG(ts_pct) AS avg_ts_pct,
MAX(ts_pct) AS max_ts_pct,

MIN(ws) AS min_ws,
AVG(ws) AS avg_ws,
MAX(ws) AS max_ws,

MIN(award_share) AS min_award_share,
AVG(award_share) AS avg_award_share,
MAX(award_share) AS max_award_share

FROM mvp_stats_gold;

min_pts_per_g,avg_pts_per_g,max_pts_per_g,min_trb_per_g,avg_trb_per_g,max_trb_per_g,min_ast_per_g,avg_ast_per_g,max_ast_per_g,min_stl_per_g,avg_stl_per_g,max_stl_per_g,min_blk_per_g,avg_blk_per_g,max_blk_per_g,min_tov_per_g,avg_tov_per_g,max_tov_per_g,min_fg_pct,avg_fg_pct,max_fg_pct,min_fg3_pct,avg_fg3_pct,max_fg3_pct,min_ft_pct,avg_ft_pct,max_ft_pct,min_per,avg_per,max_per,min_ts_pct,avg_ts_pct,max_ts_pct,min_ws,avg_ws,max_ws,min_award_share,avg_award_share,max_award_share
15.5,27.0,35.0,3.3,8.88048780487805,15.3,1.3,6.358536585365852,12.8,0.5,1.568292682926829,3.2,0.1,1.0780487804878052,3.7,2.1,3.253658536585366,5.4,0.42,0.5093902439024391,0.583,0.0,0.29417073170731706,0.454,0.524,0.813048780487805,0.921,22.0,28.03170731707317,32.8,0.518,0.5959512195121951,0.669,9.6,15.895121951219519,21.2,0.691,0.8964634146341466,1.0


#### 6.1.2 Interpretation

The summary statistics from the `mvp_stats_gold` table provide a clear profile of how MVPs perform on average across the 1982–2022 period.  
Although MVPs differ in style and position, several performance metrics show strong and consistent patterns.

##### a. Scoring is consistently high among MVPs  
- Average points per game (PPG): ~27  
- Even the lowest value (15.5 PPG) reflects an exceptional season for a non-scoring-focused player.  
This indicates that MVPs are almost always elite offensive contributors.

##### b. Efficiency metrics (PER and TS%) are universally strong  
- Player Efficiency Rating (PER) averages around 28, with all seasons above 22.  
- True Shooting(TS)% averages around 0.596.  
Both values are substantially above league averages, showing that MVPs combine volume with efficiency.

##### c. Win Shares (WS) confirm a high level of overall impact  
- Average WS is approximately 16, with a maximum exceeding 21.  
This suggests that MVPs consistently contribute heavily to team success across offence and defence.

##### d. Box score statistics vary more by position  
- Rebounds and assists show wide ranges:
  - Rebounds: from 3.3 to 15.3  
  - Assists: from 1.3 to 12.8  
These metrics depend strongly on the player’s role (e.g., guards vs. centres), meaning they are not universal indicators of MVP selection.

##### e. Turnovers are moderately high  
- Average TOV is around 3.25  
This reflects the typical MVP profile: high usage and strong involvement in ball-handling and playmaking.


#### 6.1.3 Conclusion
The statistics that most consistently appear among MVP winners are:

- High scoring (PPG)  
- High efficiency rating (PER)
- High true shooting percentage (TS%)  
- High win shares (WS)
- Moderate to high turnovers, reflecting heavy offensive responsibility  

Other statistics — such as rebounds, assists, steals and blocks — depend heavily on the player’s position and therefore do not show the same level of consistency across MVP seasons.

### 6.2 Question: _What is the distribution of player positions among MVP winners?_

#### 6.2.1 Overview and queries
To answer this question, we analyse the `pos` field in the `mvp_stats_gold` table, counting how many MVP awards were won by players at each position.
Because the Gold layer already standardised positions (C, PF, SF, SG, PG), a simple aggregation is sufficient to reveal the positional distribution.



In [0]:
%sql
SELECT pos, COUNT(*) AS mvp_count
FROM mvp_stats_gold
GROUP BY pos
ORDER BY mvp_count DESC;

pos,mvp_count
PF,11
PG,9
SG,8
C,7
SF,6


#### 6.2.2 Interpretation

The distribution of MVP awards by position shows that power forwards (PF) have been the most frequently selected MVPs in the 1982–2022 period, with 11 awards. This is followed by point guards (PG) with 9 awards and shooting guards (SG) with 8 awards. Centres (C) and small forwards (SF) complete the distribution with 7 and 6 awards respectively.

Although no single position overwhelmingly dominates, the results indicate a moderate tendency towards frontcourt players (PF and C), who together account for almost half of all MVP awards in the dataset. Point guards also feature strongly, reflecting the increasing importance of ball-handling and playmaking roles in modern NBA offence.

Meanwhile, small forwards appear the least represented, which may reflect the scarcity of historically dominant SFs in this specific period, aside from exceptional players such as Larry Bird and LeBron James.

Overall, the distribution suggests that MVP selection is not restricted to a single archetype. Instead, elite contribution can arise from different positions depending on the style of play and tactical demands of each era.


#### 6.2.3 Conclusion

MVP awards are relatively well distributed across positions, but power forwards hold a slight lead.  

Frontcourt players (PF and C) collectively appear most often, followed closely by guards (PG and SG).  

This indicates that while certain roles may offer advantages in MVP voting, exceptional players from any position can achieve MVP-level impact.

### 6.3 Question: _Do some teams historically concentrate more MVP awards?_

#### 6.3.1 Overview and queries

To answer this question, we analyse the `mvp_team_name` field in the `mvp_stats_gold` table. Each record corresponds to the team for which the MVP played in that season.

By counting the number of MVP awards per team, we can identify which franchises have historically accumulated more MVP winners during the 1982–2022 period.

This aggregation will later support a short interpretation about long-term team trends and possible historical or structural factors.

In [0]:
%sql
SELECT mvp_team_name, COUNT(*) AS mvp_count
FROM mvp_stats_gold
GROUP BY mvp_team_name
ORDER BY mvp_count DESC;

mvp_team_name,mvp_count
Chicago Bulls,6
Los Angeles Lakers,5
Houston Rockets,3
Boston Celtics,3
Phoenix Suns,3
San Antonio Spurs,3
Golden State Warriors,2
Miami Heat,2
Milwaukee Bucks,2
Utah Jazz,2


#### 6.3.2 Interpretation

The results indicate that MVP awards between 1982 and 2022 are unevenly distributed across NBA teams. The Chicago Bulls appear most frequently in the dataset, with six MVP awards, followed by the Los Angeles Lakers with five. A second group of franchises — including Houston, Boston, Phoenix and San Antonio — each accumulated three awards during the same period. Other teams appear with two or one MVP award, forming a long tail of less frequent occurrences.

While the dataset itself does not explain the underlying causes of these differences, some contextual factors may help interpret the pattern. Historically, several of the most represented teams experienced periods of strong competitiveness and organisational stability, which often coincide with the presence of high-impact players. For example, Chicago and Los Angeles both had multi-year stretches featuring dominant superstars and sustained playoff success, which may contribute to their higher MVP counts.

These contextual considerations do not form part of the dataset but offer plausible explanations for the observed concentration. The results therefore suggest that MVP recognition is influenced not only by individual performance but also by broader team environments that may support or amplify a player's value.

#### 6.3.3 Conclusion
Two franchises (the Chicago Bulls and the Los Angeles Lakers) stand out for concentrating a notably higher number of MVP awards within the period analysed. Most other teams appear only occasionally, resulting in an uneven distribution across the league.

At the same time, the data shows that **15 different teams** produced at least one MVP between 1982 and 2022, indicating a reasonable level of representativeness across franchises. This diversity suggests that, despite some concentration at the top, the NBA’s competitive structure still allows elite individual performance to emerge in a variety of organisational contexts.

While individual performance remains the primary criterion for MVP selection, contextual factors such as organisational stability, sustained competitiveness and the influence of specific historical eras may contribute to the observed concentration among a few franchises.

### 6.4 Question: How often does the MVP come from the team that wins the NBA championship in the same season?


#### 6.4.1 Overview and queries

To address this question, we use again the `mvp_champions_gold` table, which combines MVP information with season champions. For each season between 1982 and 2022, the table indicates whether the MVP and the NBA champion belonged to the same team.

The analysis consists of two simple aggregations:
1. Counting how many seasons show alignment between MVP and champion.
2. Listing which seasons they coincide, for further inspection.

In [0]:
%sql
SELECT
  SUM(mvp_and_champion_same_team) AS seasons_same_team,
  COUNT(*) as total_seasons
FROM mvp_champions_gold;

seasons_same_team,total_seasons
14,41


In [0]:
%sql
SELECT 
  season,
  player_name,
  mvp_team_name,
  champion_team_name
FROM mvp_champions_gold
WHERE mvp_and_champion_same_team = 1
ORDER BY season;

season,player_name,mvp_team_name,champion_team_name
1983,Moses Malone,Philadelphia 76ers,Philadelphia 76ers
1984,Larry Bird,Boston Celtics,Boston Celtics
1986,Larry Bird,Boston Celtics,Boston Celtics
1987,Magic Johnson,Los Angeles Lakers,Los Angeles Lakers
1991,Michael Jordan,Chicago Bulls,Chicago Bulls
1992,Michael Jordan,Chicago Bulls,Chicago Bulls
1994,Hakeem Olajuwon,Houston Rockets,Houston Rockets
1996,Michael Jordan,Chicago Bulls,Chicago Bulls
1998,Michael Jordan,Chicago Bulls,Chicago Bulls
2000,Shaquille O'Neal,Los Angeles Lakers,Los Angeles Lakers


#### 6.4.2 Interpretation

The results show that, out of 41 seasons analysed, the MVP and the NBA champion belonged to the same team in **14 seasons**. This represents roughly one-third of the period under study. Therefore, in most seasons, the MVP award does not align with the team that ultimately wins the championship.

The list of aligned seasons includes cases distributed across several decades, suggesting that this overlap is not restricted to any specific historical era. Although MVP winners often deliver exceptional individual performance, the relatively low rate of overlap shows that team success depends on a wider combination of factors beyond individual regular-season performance.

#### 6.4.3 Conclusion

In 14 of the 41 seasons analysed, the MVP and the NBA champion came from the same team.  

This indicates that while MVP-level performance can contribute to team success, it does not frequently translate into a championship in the same year.  

Individual excellence and collective achievement follow different pathways, and alignment between the two is the exception rather than the norm.


### 6.5 Question: _Which players stood out statistically even though they never won the MVP award?_

#### 6.5.1 Overview and queries

This analysis identifies players who delivered strong statistical seasons without winning the MVP award. To do this, we compare each non-MVP season against the typical profile of MVPs, focusing on metrics that were previously shown to be highly consistent among actual MVP winners:

- points per game (PTS/G)
- Player Efficiency Rating (PER)
- Win Shares (WS)
- True Shooting Percentage (TS%)

We first compute the average values of these metrics among MVP winners. Then, using the `nba_stats_gold` table, we select the non-MVP seasons that exceed these averages. This approach highlights players whose production aligns with MVP-level performance, based solely on the variables available in the dataset.

In [0]:
%sql
-- Create a view with the average values of MVP metrics as a baseline for comparison.
CREATE OR REPLACE VIEW mvp_baseline_gold AS
SELECT
    ROUND(AVG(pts_per_g), 1) AS avg_mvp_pts,
    ROUND(AVG(per),1) AS avg_mvp_per,
    ROUND(AVG(ws), 1) as avg_mvp_ws,
    ROUND(AVG(ts_pct),1) AS avg_mvp_ts
    FROM mvp_stats_gold;


In [0]:
%sql
--Filter the non-MVP players that exceeded all metrics calculated for mvps.
SELECT
  n.season,
  n.player_name,
  n.pts_per_g,
  n.per,
  n.ws,
  n.ts_pct
FROM nba_stats_gold as n
CROSS JOIN mvp_baseline_gold AS b
WHERE n.player_name NOT IN (
  SELECT player_name FROM mvp_stats_gold
)
AND (
  n.pts_per_g > b.avg_mvp_pts AND
  n.per > b.avg_mvp_per AND
  n.ws > b.avg_mvp_ws AND
  n.ts_pct > b.avg_mvp_ts
)
ORDER BY n.pts_per_g DESC
  


season,player_name,pts_per_g,per,ws,ts_pct


In [0]:
%sql
--Use a more relaxed approach (90% of the metrics) to filter non mvp players.
SELECT
  n.season,
  n.player_name,
  n.pts_per_g,
  n.per,
  n.ws,
  n.ts_pct
FROM nba_stats_gold as n
CROSS JOIN mvp_baseline_gold AS b
WHERE n.player_name NOT IN (
  SELECT player_name FROM mvp_stats_gold
)
AND (
  n.pts_per_g > 0.9 * b.avg_mvp_pts AND
  n.per > 0.9 * b.avg_mvp_per AND
  n.ws > 0.9 * b.avg_mvp_ws AND
  n.ts_pct > 0.9 * b.avg_mvp_ts
)
ORDER BY n.pts_per_g DESC



season,player_name,pts_per_g,per,ws,ts_pct
2003,Tracy McGrady,32.1,30.3,16.1,0.564
2009,Dwyane Wade,30.2,30.4,14.7,0.574
2006,Dwyane Wade,27.2,27.6,14.4,0.577
2005,Amar'e Stoudemire,26.0,26.6,14.6,0.617
2008,Amar'e Stoudemire,25.2,27.6,14.6,0.656
2006,Elton Brand,24.7,26.5,14.8,0.58


In [0]:
%sql
--Use an even more relaxed approach (80% of the metrics) to filter non mvp players.
SELECT
  n.season,
  n.player_name,
  n.pts_per_g,
  n.per,
  n.ws,
  n.ts_pct
FROM nba_stats_gold as n
CROSS JOIN mvp_baseline_gold AS b
WHERE n.player_name NOT IN (
  SELECT player_name FROM mvp_stats_gold
)
AND (
  n.pts_per_g > 0.8 * b.avg_mvp_pts AND
  n.per > 0.8 * b.avg_mvp_per AND
  n.ws > 0.8 * b.avg_mvp_ws AND
  n.ts_pct > 0.8 * b.avg_mvp_ts
)
ORDER BY n.pts_per_g DESC

season,player_name,pts_per_g,per,ws,ts_pct
2003,Tracy McGrady,32.1,30.3,16.1,0.564
1984,Adrian Dantley,30.6,24.6,14.6,0.652
2009,Dwyane Wade,30.2,30.4,14.7,0.574
1986,Adrian Dantley,29.8,24.6,12.8,0.629
2006,Gilbert Arenas,29.3,23.8,13.6,0.581
1990,Patrick Ewing,28.6,25.8,13.5,0.599
2018,Anthony Davis,28.1,28.9,13.7,0.612
2001,Vince Carter,27.6,25.0,12.9,0.551
2006,Dwyane Wade,27.2,27.6,14.4,0.577
1988,Clyde Drexler,27.0,24.1,13.2,0.564


#### 6.5.2 Interpretation

The objective of this analysis was to identify statistically outstanding seasons from players who never won the MVP award. The initial approach applied a strict criterion: a player-season would be classified as a “standout” only if it exceeded the MVP average in all four key metrics (PTS per GAME, PER, WS and TS%). This threshold proved too restrictive, **returning no non-MVP seasons**.  

To ensure the analysis remained meaningful while still maintaining a high standard, the threshold was gradually relaxed. Using 90% of the MVP averages as the benchmark resulted in **six** non-MVP seasons meeting all conditions. These cases represent players whose statistical production approached typical MVP levels across multiple dimensions simultaneously.

For exploratory purposes, a more relaxed threshold (80% of MVP averages) was also tested, returning **33** non-MVP seasons. Although the 80% group is larger, the 90% threshold provides a more selective and analytically relevant set of results, balancing strictness with practical interpretability.

#### 6.5.3 Conclusion

The strict criterion of matching or exceeding MVP averages across all key metrics identified no non-MVP seasons. When the benchmark was adjusted to 90% of MVP averages, only six non-MVP seasons qualified, indicating how uncommon it is for non-MVP players to reach statistical levels typical of MVP winners.  

Overall, the findings show that only a small number of non-MVP seasons exhibit statistical profiles comparable to MVP-level performance, even when moderately relaxed thresholds are applied.

These results suggest that, although many players may excel in one or two areas, truly well-rounded MVP-calibre production across multiple metrics is relatively rare among non-MVP players.